# Day 15
Find the description of the problem [here](https://adventofcode.com/2024/day/15)!

## Part 1

Puzzle input:

In [393]:
with open("input_files/day_15.txt") as input_file:
    input = input_file.read()

Test input:

In [394]:
# # Comment this cell to use the puzzle input instead of the test input
# input = """##########
# #..O..O.O#
# #......O.#
# #.OO..O.O#
# #..O@..O.#
# #O#..O...#
# #O..O..O.#
# #.OO.O.OO#
# #....O...#
# ##########

# <vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
# vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
# ><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
# <<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
# ^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
# ^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
# >^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
# <><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
# ^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
# v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^"""

Parse the input:

In [395]:
warehouse_raw, movements_raw = input.split("\n\n")
warehouse = [list(line) for line in warehouse_raw.split("\n")]
warehouse_width = len(warehouse[0])
warehouse_height = len(warehouse)
movements_raw = movements_raw.split("\n")
movements_single_line = ""
for line in movements_raw:
    movements_single_line += line
movements = list(movements_single_line)

Separate the robot, walls and boxes into their own lists:

In [396]:
robot_position = [0, 0]
walls = []
boxes = []
for y, line in enumerate(warehouse):
    for x, entity in enumerate(line):
        if entity == "@":
            robot_position = [x, y]
        elif entity == "O":
            boxes.append([x, y])
        elif entity == "#":
            walls.append([x, y])

The following function moves a given element (be it robot or box), and recursively makes the following box move, while checking at each step if it can move or not depending on the next element.

In [397]:
def move(position, movement):
    if movement == "<":
        dx, dy = (-1, 0)
    elif movement == ">":
        dx, dy = (1, 0)
    elif movement == "^":
        dx, dy = (0, -1)
    elif movement == "v":
        dx, dy = (0, 1)

    x, y = position
    new_pos = [x + dx, y + dy]
    should_move = False
    if new_pos in walls:
        return False
    elif new_pos in boxes:
        index = boxes.index(new_pos)
        should_move = move(boxes[index], movement)
        if should_move:
            position[0] += dx
            position[1] += dy
        return should_move
    else:
        position[0] += dx
        position[1] += dy
        return True

Iterate through all the movements:

In [398]:
for movement in movements:
    move(robot_position, movement)   

Get the coordinates:

In [399]:
gps_coordinates_sum = 0
for box in boxes:
    x, y = box
    gps_coordinates_sum += x + 100 * y

print(f"The sum of all boxes' GPS coordinates is {gps_coordinates_sum}.")

The sum of all boxes' GPS coordinates is 1517819.


And in the end draw the current state of the warehouse (not necessary to get the coordinates):

In [400]:
warehouse_now = [["." for _ in range(warehouse_width)] for _ in range(warehouse_height)]
for wall in walls:
    x, y = wall
    warehouse_now[y][x] = "#"
for box in boxes:
    x, y = box
    warehouse_now[y][x] = "O"
x, y = robot_position
warehouse_now[y][x] = "@"

# for line in warehouse_now:
#     print("".join(line))

## Part 2

Transform the coordinates to double the width:

In [401]:
robot_position = [0, 0]
walls = []
boxes = []
warehouse_width *= 2
for y, line in enumerate(warehouse):
    for x, entity in enumerate(line):
        if entity == "@":
            robot_position = [2 * x, y]
        elif entity == "O":
            boxes.append([2 * x, y])
        elif entity == "#":
            walls.append([2 * x, y])
            walls.append([2 * x + 1, y])

These two functions are the cursed way I ended up with:

In [402]:
def can_move(position, movement):
    x, y = position
    if movement == "^":
        dx, dy = (0, -1)
    elif movement == "v":
        dx, dy = (0, 1)

    if [x + dx, y + dy] in walls or [x + dx + 1, y + dy] in walls:
        return False
    elif [x + dx - 1, y + dy] in boxes and [x + dx + 1, y + dy] in boxes:
        index_1 = boxes.index([x + dx - 1, y + dy])
        index_2 = boxes.index([x + dx + 1, y + dy])
        would_move = can_move(boxes[index_1], movement) and can_move(boxes[index_2], movement)
        return would_move
    elif [x + dx - 1, y + dy] in boxes:
        index = boxes.index([x + dx - 1, y + dy])
        would_move = can_move(boxes[index], movement)
        return would_move
    elif [x + dx, y + dy] in boxes:
        index = boxes.index([x + dx, y + dy])
        would_move = can_move(boxes[index], movement)
        return would_move
    elif [x + dx + 1, y + dy] in boxes:
        index = boxes.index([x + dx + 1, y + dy])
        would_move = can_move(boxes[index], movement)
        return would_move
    else:
        return True

In [403]:
def move_double_width(position, movement, robot=True):
    x, y = position
    should_move = False

    if movement == "<":
        # When moving to the left, boxes are one distance further
        dx, dy = (-1, 0)
        if [x + dx, y + dy] in walls:
            return False
        elif [x + dx - 1, y + dy] in boxes:
            index = boxes.index([x + dx - 1, y + dy])
            should_move = move_double_width(boxes[index], movement, robot=False)
            if should_move:
                position[0] += dx
                position[1] += dy
            return should_move
        else:
            position[0] += dx
            position[1] += dy
            return True
    elif movement == ">":
        # When moving to the right, there are two options. If the robot moves, since it's only one tile wide, it's a normal check
        # When the moving item is a box, it needs to check one further to the right to account for its own width
        dx, dy = (1, 0)
        if robot:
            if [x + dx, y + dy] in walls:
                return False
            elif [x + dx, y + dy] in boxes:
                index = boxes.index([x + dx, y + dy])
                should_move = move_double_width(boxes[index], movement, robot=False)
                if should_move:
                    position[0] += dx
                    position[1] += dy
                return should_move
            else:
                position[0] += dx
                position[1] += dy
                return True
        else:
            if [x + dx + 1, y + dy] in walls:
                return False
            elif [x + dx + 1, y + dy] in boxes:
                index = boxes.index([x + dx + 1, y + dy])
                should_move = move_double_width(boxes[index], movement, robot=False)
                if should_move:
                    position[0] += dx
                    position[1] += dy
                return should_move
            else:
                position[0] += dx
                position[1] += dy
                return True
    elif movement == "^":
        dx, dy = (0, -1)
    elif movement == "v":
        dx, dy = (0, 1)

    # Moving up or down is the same, just with a different direction
    if robot:
        # If the robot is the one moving, it needs to check for boxes one to the left as well (when it hits the right side of the box)
        if [x + dx, y + dy] in walls:
            return False
        elif [x + dx, y + dy] in boxes:
            index = boxes.index([x + dx, y + dy])
            should_move = move_double_width(boxes[index], movement, robot=False)
            if should_move:
                position[0] += dx
                position[1] += dy
            return should_move
        elif [x + dx - 1, y + dy] in boxes:
            index = boxes.index([x + dx - 1, y + dy])
            should_move = move_double_width(boxes[index], movement, robot=False)
            if should_move:
                position[0] += dx
                position[1] += dy
            return should_move
        else:
            position[0] += dx
            position[1] += dy
            return True
    else:
        # If it's a box that is moving, it becomes hell
        if [x + dx, y + dy] in walls or [x + dx + 1, y + dy] in walls:
            return False
        elif [x + dx - 1, y + dy] in boxes and [x + dx + 1, y + dy] in boxes:
            # In the event that the box touches two boxes, both need to be checked for moveability before moving any of them
            # (hence the can_move function, which does the checking without the moving)
            index_1 = boxes.index([x + dx - 1, y + dy])
            index_2 = boxes.index([x + dx + 1, y + dy])
            should_move = can_move(boxes[index_1], movement) and can_move(boxes[index_2], movement)
            if should_move:
                move_double_width(boxes[index_1], movement, robot=False)
                move_double_width(boxes[index_2], movement, robot=False)
                position[0] += dx
                position[1] += dy
            return should_move
        elif [x + dx - 1, y + dy] in boxes:
            # Checking for the box misaligned to the left
            index = boxes.index([x + dx - 1, y + dy])
            should_move = move_double_width(boxes[index], movement, robot=False)
            if should_move:
                position[0] += dx
                position[1] += dy
            return should_move
        elif [x + dx, y + dy] in boxes:
            # Checking for the box just above/below
            index = boxes.index([x + dx, y + dy])
            should_move = move_double_width(boxes[index], movement, robot=False)
            if should_move:
                position[0] += dx
                position[1] += dy
            return should_move
        elif [x + dx + 1, y + dy] in boxes:
            # Checking for the box misaligned to the right
            index = boxes.index([x + dx + 1, y + dy])
            should_move = move_double_width(boxes[index], movement, robot=False)
            if should_move:
                position[0] += dx
                position[1] += dy
            return should_move
        else:
            position[0] += dx
            position[1] += dy
            return True

Iterate through all the movements:

In [404]:
for movement in movements:
    move_double_width(robot_position, movement)

Get the coordinates:

In [405]:
gps_coordinates_sum = 0
for box in boxes:
    x, y = box
    gps_coordinates_sum += x + 100 * y

print(f"The sum of all boxes' GPS coordinates is {gps_coordinates_sum}.")

The sum of all boxes' GPS coordinates is 1538862.


Draw the double width warehouse:

In [406]:
warehouse_now = [["." for _ in range(warehouse_width)] for _ in range(warehouse_height)]
for wall in walls:
    x, y = wall
    warehouse_now[y][x] = "#"
for box in boxes:
    x, y = box
    warehouse_now[y][x] = "["
    warehouse_now[y][x + 1] = "]"
x, y = robot_position
warehouse_now[y][x] = "@"

# for line in warehouse_now:
#     print("".join(line))